In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
import data_utils.data_loading as data_load
from data_utils.cifar10_testset import CIFAR10Testset
from custom_nets.vgglike import VGGLike

random_seed = 42
random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

In [2]:
if not os.path.exists('../tests'):
    os.makedirs('../tests')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
batch_size = 32
softmax = nn.Softmax(1)

In [4]:
transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
testloader, number_of_images = data_load.load_test_data(transform, batch_size)
model = VGGLike()
model.load_state_dict(torch.load('../nets/cifar_net_Adam001_20220326_132429'))
model.to(device)

labels = np.empty((number_of_images, 10))
total = 0
with torch.no_grad():
    for data in testloader:
        images = data[0].to(device)
        outputs = model(images)
        labels[total:(total + len(outputs)), :] = softmax(outputs).data.detach().cpu().numpy()
        total += len(outputs)
del model
del testloader

In [6]:
transform2 = transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
testloader2, _ = data_load.load_test_data(transform2, batch_size)

In [7]:
model2 = torchvision.models.resnet18(pretrained=True)
num_ftrs = model2.fc.in_features
model2.fc = nn.Linear(num_ftrs, 10)
model2.load_state_dict(torch.load('../nets/cifar_resnet18_Adam001_20220324_143450'))
model2.to(device)

labels2 = np.empty((number_of_images, 10))
total = 0
with torch.no_grad():
    for data in testloader2:
        images = data[0].to(device)
        outputs2 = model2(images)
        labels2[total:(total + len(outputs2)), :] = softmax(outputs2).data.detach().cpu().numpy()
        total += len(outputs2)
del model2

In [8]:
model3 = torchvision.models.efficientnet_b0(pretrained=True)
num_ftrs = model3.classifier[1].in_features
model3.classifier[1] = nn.Linear(num_ftrs, 10)
model3.load_state_dict(torch.load('../nets/cifar_efficientnetb0_SGD0019_20220323_141233'))
model3.to(device)

labels3 = np.empty((number_of_images, 10))
total = 0
with torch.no_grad():
    for data in testloader2:
        images = data[0].to(device)
        outputs3 = model3(images)
        labels3[total:(total + len(outputs3)), :] = softmax(outputs3).data.detach().cpu().numpy()
        total += len(outputs3)
del testloader2
del model3

In [9]:
ensemble = labels + labels2 + labels3
final_labels = np.argmax(ensemble, axis=1)
classes = np.empty(number_of_images, dtype=object)
for i, label in enumerate(final_labels):
    classes[i] = CIFAR10Testset.label_number_to_class[label]
pd.DataFrame(classes, index=range(1, number_of_images + 1), columns=['label']).to_csv('../tests/ensemble.csv', index_label='id')